In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, f1_score
import lightgbm as lgb
from collections import Counter
import warnings
import gc
# from pandasgui import show
warnings.filterwarnings("ignore")

In [3]:
pd.set_option('display.max_rows',5)   # 具体的行数或列数可自行设置
pd.set_option('display.max_columns',100)

In [4]:
def get_base_info(x):
    return [i.split(':')[-1] for i in x.split(' ')]

def get_speed(x):
    return np.array([i.split(',')[0] for i in x], dtype='float16')

def get_eta(x):
    return np.array([i.split(',')[1] for i in x], dtype='float16')

def get_state(x):
    return np.array([(i.split(',')[2]) for i in x], dtype='int16')

def get_cnt(x):
    return np.array([i.split(',')[3] for i in x], dtype='int16')

In [5]:
path = 'test.txt'

In [6]:
df = pd.read_csv(path, sep=';', header=None)

In [7]:
df['link'] = df[0].apply(lambda x: x.split(' ')[0])
df['label'] = -1
df['current_slice_id'] = df[0].apply(lambda x: int(x.split(' ')[2]))
df['future_slice_id'] = df[0].apply(lambda x: int(x.split(' ')[3]))

In [8]:
df['time_diff'] = df['future_slice_id'] - df['current_slice_id']

In [9]:
df

,0,1,2,3,4,5,link,label,current_slice_id,future_slice_id,time_diff
0,106984 -1 616 640,"612:28.10,36.80,1,21 613:30.20,36.80,1,19 614:...","640:28.80,40.90,1,18 641:37.80,41.20,1,21 642:...","640:29.10,39.50,1,37 641:30.00,40.50,1,41 642:...","640:30.30,37.40,1,37 641:33.80,37.70,1,38 642:...","640:30.80,36.20,1,39 641:31.60,36.30,1,32 642:...",106984,-1,616,640,24
1,33252 -1 327 343,"323:23.30,23.00,2,2 324:22.90,22.40,2,1 325:22...","343:28.30,25.10,1,3 344:27.10,25.10,1,3 345:27...","343:32.20,24.10,1,5 344:31.20,24.50,1,6 345:34...","343:33.20,33.30,1,5 344:33.60,29.30,1,5 345:29...","343:31.70,19.10,1,4 344:31.70,20.80,1,5 345:32...",33252,-1,327,343,16
...,...,...,...,...,...,...,...,...,...,...,...
504889,359102 -1 451 459,"447:52.50,47.30,1,103 448:48.70,44.00,1,106 44...","459:43.90,41.30,1,94 460:45.50,40.80,1,107 461...","459:49.40,38.30,1,120 460:46.80,40.30,1,118 46...","459:43.50,39.20,1,111 460:43.60,40.90,1,113 46...","459:48.70,46.40,1,107 460:45.30,45.20,1,96 461...",359102,-1,451,459,8
504890,226212 -1 378 385,"374:34.90,48.40,1,2 375:28.80,19.90,1,2 376:28...","385:40.60,44.20,1,2 386:38.50,44.20,1,2 387:38...","385:34.80,38.80,1,4 386:36.70,52.60,1,2 387:40...","385:35.70,29.30,1,1 386:29.90,25.50,1,2 387:37...","385:34.80,34.80,1,2 386:34.80,34.80,1,2 387:30...",226212,-1,378,385,7


In [10]:
gc.collect()

20

In [11]:
df['current_slice_id'] = df['current_slice_id'].apply(lambda x: 1 if x <= 200 else x)
df['current_slice_id'] = df['current_slice_id'].apply(lambda x: 2 if (200< x <= 230) else x)
df['current_slice_id'] = df['current_slice_id'].apply(lambda x: 3 if (230< x <= 340) else x)
df['current_slice_id'] = df['current_slice_id'].apply(lambda x: 4 if (340< x <= 470) else x)
df['current_slice_id'] = df['current_slice_id'].apply(lambda x: 5 if (470< x <= 520) else x)
df['current_slice_id'] = df['current_slice_id'].apply(lambda x: 6 if (520< x <= 588) else x)
df['current_slice_id'] = df['current_slice_id'].apply(lambda x: 7 if (588< x <= 720) else x)

In [12]:
df['future_slice_id'] = df['future_slice_id'].apply(lambda x: 1 if x <= 200 else x)
df['future_slice_id'] = df['future_slice_id'].apply(lambda x: 2 if (200< x <= 230) else x)
df['future_slice_id'] = df['future_slice_id'].apply(lambda x: 3 if (230< x <= 340) else x)
df['future_slice_id'] = df['future_slice_id'].apply(lambda x: 4 if (340< x <= 470) else x)
df['future_slice_id'] = df['future_slice_id'].apply(lambda x: 5 if (470< x <= 520) else x)
df['future_slice_id'] = df['future_slice_id'].apply(lambda x: 6 if (520< x <= 588) else x)
df['future_slice_id'] = df['future_slice_id'].apply(lambda x: 7 if (588< x <= 720) else x)

In [13]:
df

,0,1,2,3,4,5,link,label,current_slice_id,future_slice_id,time_diff
0,106984 -1 616 640,"612:28.10,36.80,1,21 613:30.20,36.80,1,19 614:...","640:28.80,40.90,1,18 641:37.80,41.20,1,21 642:...","640:29.10,39.50,1,37 641:30.00,40.50,1,41 642:...","640:30.30,37.40,1,37 641:33.80,37.70,1,38 642:...","640:30.80,36.20,1,39 641:31.60,36.30,1,32 642:...",106984,-1,7,7,24
1,33252 -1 327 343,"323:23.30,23.00,2,2 324:22.90,22.40,2,1 325:22...","343:28.30,25.10,1,3 344:27.10,25.10,1,3 345:27...","343:32.20,24.10,1,5 344:31.20,24.50,1,6 345:34...","343:33.20,33.30,1,5 344:33.60,29.30,1,5 345:29...","343:31.70,19.10,1,4 344:31.70,20.80,1,5 345:32...",33252,-1,3,4,16
...,...,...,...,...,...,...,...,...,...,...,...
504889,359102 -1 451 459,"447:52.50,47.30,1,103 448:48.70,44.00,1,106 44...","459:43.90,41.30,1,94 460:45.50,40.80,1,107 461...","459:49.40,38.30,1,120 460:46.80,40.30,1,118 46...","459:43.50,39.20,1,111 460:43.60,40.90,1,113 46...","459:48.70,46.40,1,107 460:45.30,45.20,1,96 461...",359102,-1,4,4,8
504890,226212 -1 378 385,"374:34.90,48.40,1,2 375:28.80,19.90,1,2 376:28...","385:40.60,44.20,1,2 386:38.50,44.20,1,2 387:38...","385:34.80,38.80,1,4 386:36.70,52.60,1,2 387:40...","385:35.70,29.30,1,1 386:29.90,25.50,1,2 387:37...","385:34.80,34.80,1,2 386:34.80,34.80,1,2 387:30...",226212,-1,4,4,7


In [14]:
for i in tqdm(range(1, 6)):
        df['his_info'] = df[i].apply(get_base_info)
        if i == 1:
            flg = 'current'
        else:
            flg = f'his_{(6 - i) * 7}'
        df['his_speed'] = df['his_info'].apply(get_speed)
        df[f'{flg}_speed_min'] = df['his_speed'].apply(lambda x: x.min())
        df[f'{flg}_speed_max'] = df['his_speed'].apply(lambda x: x.max())
        df[f'{flg}_speed_mean'] = df['his_speed'].apply(lambda x: x.mean())
        df[f'{flg}_speed_std'] = df['his_speed'].apply(lambda x: x.std())

        df['his_eta'] = df['his_info'].apply(get_eta)
        df[f'{flg}_eta_min'] = df['his_eta'].apply(lambda x: x.min())
        df[f'{flg}_eta_max'] = df['his_eta'].apply(lambda x: x.max())
        df[f'{flg}_eta_mean'] = df['his_eta'].apply(lambda x: x.mean())
        df[f'{flg}_eta_std'] = df['his_eta'].apply(lambda x: x.std())

        df['his_cnt'] = df['his_info'].apply(get_cnt)
        df[f'{flg}_cnt_min'] = df['his_cnt'].apply(lambda x: x.min())
        df[f'{flg}_cnt_max'] = df['his_cnt'].apply(lambda x: x.max())
        df[f'{flg}_cnt_mean'] = df['his_cnt'].apply(lambda x: x.mean())
        df[f'{flg}_cnt_std'] = df['his_cnt'].apply(lambda x: x.std())
        
        df['his_state'] = df['his_info'].apply(get_state)
        df[f'{flg}__state_min'] = df['his_state'].apply(lambda x: x.min())
        df[f'{flg}__state_max'] = df['his_state'].apply(lambda x: x.max())
        df[f'{flg}_state'] = df['his_state'].apply(lambda x: Counter(x).most_common()[0][0])
        df.drop([i, 'his_info', 'his_speed', 'his_eta', 'his_cnt', 'his_state'], axis=1, inplace=True)
        #df.drop([i, 'his_info', 'his_speed', 'his_eta', 'his_cnt'], axis=1, inplace=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [04:46<00:00, 57.24s/it]


In [15]:
df

,0,link,label,current_slice_id,future_slice_id,time_diff,current_speed_min,current_speed_max,current_speed_mean,current_speed_std,current_eta_min,current_eta_max,current_eta_mean,current_eta_std,current_cnt_min,current_cnt_max,current_cnt_mean,current_cnt_std,current__state_min,current__state_max,current_state,his_28_speed_min,his_28_speed_max,his_28_speed_mean,his_28_speed_std,his_28_eta_min,his_28_eta_max,his_28_eta_mean,his_28_eta_std,his_28_cnt_min,his_28_cnt_max,his_28_cnt_mean,his_28_cnt_std,his_28__state_min,his_28__state_max,his_28_state,his_21_speed_min,his_21_speed_max,his_21_speed_mean,his_21_speed_std,his_21_eta_min,his_21_eta_max,his_21_eta_mean,his_21_eta_std,his_21_cnt_min,his_21_cnt_max,his_21_cnt_mean,his_21_cnt_std,his_21__state_min,his_21__state_max,his_21_state,his_14_speed_min,his_14_speed_max,his_14_speed_mean,his_14_speed_std,his_14_eta_min,his_14_eta_max,his_14_eta_mean,his_14_eta_std,his_14_cnt_min,his_14_cnt_max,his_14_cnt_mean,his_14_cnt_std,his_14__state_min,his_14__state_max,his_14_state,his_7_speed_min,his_7_speed_max,his_7_speed_mean,his_7_speed_std,his_7_eta_min,his_7_eta_max,his_7_eta_mean,his_7_eta_std,his_7_cnt_min,his_7_cnt_max,his_7_cnt_mean,his_7_cnt_std,his_7__state_min,his_7__state_max,his_7_state
0,106984 -1 616 640,106984,-1,7,7,24,28.093750,30.203125,29.156250,0.859375,36.81250,38.59375,37.875000,0.858398,19,24,21.6,1.743560,1,1,1,28.09375,37.81250,30.843750,3.660156,38.68750,41.187500,40.12500,0.885254,18,27,23.0,3.521363,1,1,1,28.906250,30.203125,29.578125,0.503906,38.00000,40.50000,39.093750,0.846191,31,42,37.2,4.019950,1,1,1,30.296875,33.81250,31.328125,1.301758,34.68750,38.31250,37.15625,1.269531,35,38,37.0,1.095445,1,1,1,28.406250,31.59375,29.953125,1.252930,36.09375,39.09375,37.031250,1.147461,30,39,33.0,3.162278,1,1,1
1,33252 -1 327 343,33252,-1,3,4,16,22.906250,30.000000,25.828125,3.416016,22.40625,23.00000,22.671875,0.203003,1,2,1.8,0.400000,0,2,2,27.09375,35.18750,28.953125,3.152344,25.09375,29.703125,27.34375,1.894531,2,3,2.8,0.400000,1,1,1,31.203125,36.187500,33.906250,1.907227,24.09375,33.18750,28.984375,3.900391,4,6,5.2,0.748331,1,1,1,28.296875,33.59375,30.875000,2.144531,27.40625,33.31250,29.37500,2.056641,5,7,6.2,0.979796,1,1,1,30.593750,37.50000,32.750000,2.437500,19.09375,23.09375,20.921875,1.273438,4,6,5.0,0.632456,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504889,359102 -1 451 459,359102,-1,4,4,8,45.000000,52.500000,48.250000,2.451172,44.00000,47.68750,46.187500,1.298828,94,106,100.6,4.715930,1,1,1,43.90625,45.50000,44.562500,0.627930,40.81250,42.593750,41.37500,0.651855,94,111,104.0,5.932959,1,1,1,46.812500,49.406250,47.718750,0.927734,38.31250,45.00000,41.437500,2.564453,110,124,117.8,4.578209,1,1,1,42.906250,43.59375,43.281250,0.275879,39.18750,41.90625,41.06250,1.000977,93,113,105.2,7.166589,1,1,1,41.812500,48.68750,44.937500,2.216797,40.81250,46.40625,43.593750,2.298828,96,107,99.2,4.166533,1,1,1
504890,226212 -1 378 385,226212,-1,4,4,7,28.703125,34.906250,30.218750,2.382812,19.40625,48.40625,25.406250,11.500000,2,3,2.4,0.489898,1,1,1,38.50000,40.59375,38.906250,0.837891,44.18750,44.187500,44.18750,0.000000,2,2,2.0,0.000000,1,1,1,34.812500,41.093750,38.750000,2.541016,38.81250,52.59375,43.968750,4.582031,1,4,1.8,1.166190,1,1,1,29.906250,37.90625,34.593750,2.875000,25.50000,45.50000,36.59375,8.187500,1,4,2.6,1.019804,1,1,1,28.203125,34.81250,31.312500,2.982422,24.00000,34.81250,28.328125,5.300781,1,2,1.4,0.489898,1,1,1


In [16]:
attr = pd.read_csv('attr.txt', sep='\t',
                       names=['link', 'length', 'direction', 'path_class', 'speed_class', 'LaneNum', 'speed_limit',
                              'level', 'width'], header=None)

In [17]:
df['link'] = df['link'].apply(int)

In [18]:
del df[0]

In [19]:
test = df.merge(attr, on='link', how='left')

In [20]:
test

,link,label,current_slice_id,future_slice_id,time_diff,current_speed_min,current_speed_max,current_speed_mean,current_speed_std,current_eta_min,current_eta_max,current_eta_mean,current_eta_std,current_cnt_min,current_cnt_max,current_cnt_mean,current_cnt_std,current__state_min,current__state_max,current_state,his_28_speed_min,his_28_speed_max,his_28_speed_mean,his_28_speed_std,his_28_eta_min,his_28_eta_max,his_28_eta_mean,his_28_eta_std,his_28_cnt_min,his_28_cnt_max,his_28_cnt_mean,his_28_cnt_std,his_28__state_min,his_28__state_max,his_28_state,his_21_speed_min,his_21_speed_max,his_21_speed_mean,his_21_speed_std,his_21_eta_min,his_21_eta_max,his_21_eta_mean,his_21_eta_std,his_21_cnt_min,his_21_cnt_max,his_21_cnt_mean,his_21_cnt_std,his_21__state_min,his_21__state_max,his_21_state,his_14_speed_min,his_14_speed_max,his_14_speed_mean,his_14_speed_std,his_14_eta_min,his_14_eta_max,his_14_eta_mean,his_14_eta_std,his_14_cnt_min,his_14_cnt_max,his_14_cnt_mean,his_14_cnt_std,his_14__state_min,his_14__state_max,his_14_state,his_7_speed_min,his_7_speed_max,his_7_speed_mean,his_7_speed_std,his_7_eta_min,his_7_eta_max,his_7_eta_mean,his_7_eta_std,his_7_cnt_min,his_7_cnt_max,his_7_cnt_mean,his_7_cnt_std,his_7__state_min,his_7__state_max,his_7_state,length,direction,path_class,speed_class,LaneNum,speed_limit,level,width
0,106984,-1,7,7,24,28.093750,30.203125,29.156250,0.859375,36.81250,38.59375,37.875000,0.858398,19,24,21.6,1.743560,1,1,1,28.09375,37.81250,30.843750,3.660156,38.68750,41.187500,40.12500,0.885254,18,27,23.0,3.521363,1,1,1,28.906250,30.203125,29.578125,0.503906,38.00000,40.50000,39.093750,0.846191,31,42,37.2,4.019950,1,1,1,30.296875,33.81250,31.328125,1.301758,34.68750,38.31250,37.15625,1.269531,35,38,37.0,1.095445,1,1,1,28.406250,31.59375,29.953125,1.252930,36.09375,39.09375,37.031250,1.147461,30,39,33.0,3.162278,1,1,1,40,3,4,5,2,16.666667,4,55
1,33252,-1,3,4,16,22.906250,30.000000,25.828125,3.416016,22.40625,23.00000,22.671875,0.203003,1,2,1.8,0.400000,0,2,2,27.09375,35.18750,28.953125,3.152344,25.09375,29.703125,27.34375,1.894531,2,3,2.8,0.400000,1,1,1,31.203125,36.187500,33.906250,1.907227,24.09375,33.18750,28.984375,3.900391,4,6,5.2,0.748331,1,1,1,28.296875,33.59375,30.875000,2.144531,27.40625,33.31250,29.37500,2.056641,5,7,6.2,0.979796,1,1,1,30.593750,37.50000,32.750000,2.437500,19.09375,23.09375,20.921875,1.273438,4,6,5.0,0.632456,1,1,1,125,2,5,5,2,16.666667,5,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504889,359102,-1,4,4,8,45.000000,52.500000,48.250000,2.451172,44.00000,47.68750,46.187500,1.298828,94,106,100.6,4.715930,1,1,1,43.90625,45.50000,44.562500,0.627930,40.81250,42.593750,41.37500,0.651855,94,111,104.0,5.932959,1,1,1,46.812500,49.406250,47.718750,0.927734,38.31250,45.00000,41.437500,2.564453,110,124,117.8,4.578209,1,1,1,42.906250,43.59375,43.281250,0.275879,39.18750,41.90625,41.06250,1.000977,93,113,105.2,7.166589,1,1,1,41.812500,48.68750,44.937500,2.216797,40.81250,46.40625,43.593750,2.298828,96,107,99.2,4.166533,1,1,1,23,3,2,5,2,19.444444,2,55
504890,226212,-1,4,4,7,28.703125,34.906250,30.218750,2.382812,19.40625,48.40625,25.406250,11.500000,2,3,2.4,0.489898,1,1,1,38.50000,40.59375,38.906250,0.837891,44.18750,44.187500,44.18750,0.000000,2,2,2.0,0.000000,1,1,1,34.812500,41.093750,38.750000,2.541016,38.81250,52.59375,43.968750,4.582031,1,4,1.8,1.166190,1,1,1,29.906250,37.90625,34.593750,2.875000,25.50000,45.50000,36.59375,8.187500,1,4,2.6,1.019804,1,1,1,28.203125,34.81250,31.312500,2.982422,24.00000,34.81250,28.328125,5.300781,1,2,1.4,0.489898,1,1,1,40,1,5,6,1,11.111111,5,55


In [31]:
test

,label,current_slice_id,future_slice_id,time_diff,current_speed_min,current_speed_max,current_speed_mean,current_speed_std,current_eta_min,current_eta_max,current_eta_mean,current_eta_std,current_cnt_min,current_cnt_max,current_cnt_mean,current_cnt_std,current__state_min,current__state_max,current_state,his_28_speed_min,his_28_speed_max,his_28_speed_mean,his_28_speed_std,his_28_eta_min,his_28_eta_max,his_28_eta_mean,his_28_eta_std,his_28_cnt_min,his_28_cnt_max,his_28_cnt_mean,his_28_cnt_std,his_28__state_min,his_28__state_max,his_28_state,his_21_speed_min,his_21_speed_max,his_21_speed_mean,his_21_speed_std,his_21_eta_min,his_21_eta_max,his_21_eta_mean,his_21_eta_std,his_21_cnt_min,his_21_cnt_max,his_21_cnt_mean,his_21_cnt_std,his_21__state_min,his_21__state_max,his_21_state,his_14_speed_min,his_14_speed_max,his_14_speed_mean,his_14_speed_std,his_14_eta_min,his_14_eta_max,his_14_eta_mean,his_14_eta_std,his_14_cnt_min,his_14_cnt_max,his_14_cnt_mean,his_14_cnt_std,his_14__state_min,his_14__state_max,his_14_state,his_7_speed_min,his_7_speed_max,his_7_speed_mean,his_7_speed_std,his_7_eta_min,his_7_eta_max,his_7_eta_mean,his_7_eta_std,his_7_cnt_min,his_7_cnt_max,his_7_cnt_mean,his_7_cnt_std,his_7__state_min,his_7__state_max,his_7_state,link,length,direction,path_class,speed_class,LaneNum,speed_limit,level,width
0,-1,7,7,24,28.093750,30.203125,29.156250,0.859375,36.81250,38.59375,37.875000,0.858398,19,24,21.6,1.743560,1,1,1,28.09375,37.81250,30.843750,3.660156,38.68750,41.187500,40.12500,0.885254,18,27,23.0,3.521363,1,1,1,28.906250,30.203125,29.578125,0.503906,38.00000,40.50000,39.093750,0.846191,31,42,37.2,4.019950,1,1,1,30.296875,33.81250,31.328125,1.301758,34.68750,38.31250,37.15625,1.269531,35,38,37.0,1.095445,1,1,1,28.406250,31.59375,29.953125,1.252930,36.09375,39.09375,37.031250,1.147461,30,39,33.0,3.162278,1,1,1,106984,40,3,4,5,2,16.666667,4,55
1,-1,3,4,16,22.906250,30.000000,25.828125,3.416016,22.40625,23.00000,22.671875,0.203003,1,2,1.8,0.400000,0,2,2,27.09375,35.18750,28.953125,3.152344,25.09375,29.703125,27.34375,1.894531,2,3,2.8,0.400000,1,1,1,31.203125,36.187500,33.906250,1.907227,24.09375,33.18750,28.984375,3.900391,4,6,5.2,0.748331,1,1,1,28.296875,33.59375,30.875000,2.144531,27.40625,33.31250,29.37500,2.056641,5,7,6.2,0.979796,1,1,1,30.593750,37.50000,32.750000,2.437500,19.09375,23.09375,20.921875,1.273438,4,6,5.0,0.632456,1,1,1,33252,125,2,5,5,2,16.666667,5,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504889,-1,4,4,8,45.000000,52.500000,48.250000,2.451172,44.00000,47.68750,46.187500,1.298828,94,106,100.6,4.715930,1,1,1,43.90625,45.50000,44.562500,0.627930,40.81250,42.593750,41.37500,0.651855,94,111,104.0,5.932959,1,1,1,46.812500,49.406250,47.718750,0.927734,38.31250,45.00000,41.437500,2.564453,110,124,117.8,4.578209,1,1,1,42.906250,43.59375,43.281250,0.275879,39.18750,41.90625,41.06250,1.000977,93,113,105.2,7.166589,1,1,1,41.812500,48.68750,44.937500,2.216797,40.81250,46.40625,43.593750,2.298828,96,107,99.2,4.166533,1,1,1,359102,23,3,2,5,2,19.444444,2,55
504890,-1,4,4,7,28.703125,34.906250,30.218750,2.382812,19.40625,48.40625,25.406250,11.500000,2,3,2.4,0.489898,1,1,1,38.50000,40.59375,38.906250,0.837891,44.18750,44.187500,44.18750,0.000000,2,2,2.0,0.000000,1,1,1,34.812500,41.093750,38.750000,2.541016,38.81250,52.59375,43.968750,4.582031,1,4,1.8,1.166190,1,1,1,29.906250,37.90625,34.593750,2.875000,25.50000,45.50000,36.59375,8.187500,1,4,2.6,1.019804,1,1,1,28.203125,34.81250,31.312500,2.982422,24.00000,34.81250,28.328125,5.300781,1,2,1.4,0.489898,1,1,1,226212,40,1,5,6,1,11.111111,5,55


In [21]:
import pickle
with open('knn.txt','rb')as f:
    clf = pickle.load(f)

In [22]:
use_cols = [i for i in test.columns if i not in ['link', 'label', 'label_pred','level']]

In [23]:
X_test = test[use_cols]

In [39]:
X_test

,current_slice_id,future_slice_id,time_diff,current_speed_min,current_speed_max,current_speed_mean,current_speed_std,current_eta_min,current_eta_max,current_eta_mean,current_eta_std,current_cnt_min,current_cnt_max,current_cnt_mean,current_cnt_std,current__state_min,current__state_max,current_state,his_28_speed_min,his_28_speed_max,his_28_speed_mean,his_28_speed_std,his_28_eta_min,his_28_eta_max,his_28_eta_mean,his_28_eta_std,his_28_cnt_min,his_28_cnt_max,his_28_cnt_mean,his_28_cnt_std,his_28__state_min,his_28__state_max,his_28_state,his_21_speed_min,his_21_speed_max,his_21_speed_mean,his_21_speed_std,his_21_eta_min,his_21_eta_max,his_21_eta_mean,his_21_eta_std,his_21_cnt_min,his_21_cnt_max,his_21_cnt_mean,his_21_cnt_std,his_21__state_min,his_21__state_max,his_21_state,his_14_speed_min,his_14_speed_max,his_14_speed_mean,his_14_speed_std,his_14_eta_min,his_14_eta_max,his_14_eta_mean,his_14_eta_std,his_14_cnt_min,his_14_cnt_max,his_14_cnt_mean,his_14_cnt_std,his_14__state_min,his_14__state_max,his_14_state,his_7_speed_min,his_7_speed_max,his_7_speed_mean,his_7_speed_std,his_7_eta_min,his_7_eta_max,his_7_eta_mean,his_7_eta_std,his_7_cnt_min,his_7_cnt_max,his_7_cnt_mean,his_7_cnt_std,his_7__state_min,his_7__state_max,his_7_state,length,direction,path_class,speed_class,LaneNum,speed_limit,width
0,7,7,24,28.093750,30.203125,29.156250,0.859375,36.81250,38.59375,37.875000,0.858398,19,24,21.6,1.743560,1,1,1,28.09375,37.81250,30.843750,3.660156,38.68750,41.187500,40.12500,0.885254,18,27,23.0,3.521363,1,1,1,28.906250,30.203125,29.578125,0.503906,38.00000,40.50000,39.093750,0.846191,31,42,37.2,4.019950,1,1,1,30.296875,33.81250,31.328125,1.301758,34.68750,38.31250,37.15625,1.269531,35,38,37.0,1.095445,1,1,1,28.406250,31.59375,29.953125,1.252930,36.09375,39.09375,37.031250,1.147461,30,39,33.0,3.162278,1,1,1,40,3,4,5,2,16.666667,55
1,3,4,16,22.906250,30.000000,25.828125,3.416016,22.40625,23.00000,22.671875,0.203003,1,2,1.8,0.400000,0,2,2,27.09375,35.18750,28.953125,3.152344,25.09375,29.703125,27.34375,1.894531,2,3,2.8,0.400000,1,1,1,31.203125,36.187500,33.906250,1.907227,24.09375,33.18750,28.984375,3.900391,4,6,5.2,0.748331,1,1,1,28.296875,33.59375,30.875000,2.144531,27.40625,33.31250,29.37500,2.056641,5,7,6.2,0.979796,1,1,1,30.593750,37.50000,32.750000,2.437500,19.09375,23.09375,20.921875,1.273438,4,6,5.0,0.632456,1,1,1,125,2,5,5,2,16.666667,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504889,4,4,8,45.000000,52.500000,48.250000,2.451172,44.00000,47.68750,46.187500,1.298828,94,106,100.6,4.715930,1,1,1,43.90625,45.50000,44.562500,0.627930,40.81250,42.593750,41.37500,0.651855,94,111,104.0,5.932959,1,1,1,46.812500,49.406250,47.718750,0.927734,38.31250,45.00000,41.437500,2.564453,110,124,117.8,4.578209,1,1,1,42.906250,43.59375,43.281250,0.275879,39.18750,41.90625,41.06250,1.000977,93,113,105.2,7.166589,1,1,1,41.812500,48.68750,44.937500,2.216797,40.81250,46.40625,43.593750,2.298828,96,107,99.2,4.166533,1,1,1,23,3,2,5,2,19.444444,55
504890,4,4,7,28.703125,34.906250,30.218750,2.382812,19.40625,48.40625,25.406250,11.500000,2,3,2.4,0.489898,1,1,1,38.50000,40.59375,38.906250,0.837891,44.18750,44.187500,44.18750,0.000000,2,2,2.0,0.000000,1,1,1,34.812500,41.093750,38.750000,2.541016,38.81250,52.59375,43.968750,4.582031,1,4,1.8,1.166190,1,1,1,29.906250,37.90625,34.593750,2.875000,25.50000,45.50000,36.59375,8.187500,1,4,2.6,1.019804,1,1,1,28.203125,34.81250,31.312500,2.982422,24.00000,34.81250,28.328125,5.300781,1,2,1.4,0.489898,1,1,1,40,1,5,6,1,11.111111,55


In [24]:
X_test = np.array(X_test)

In [25]:
clf.predict(X_test[504879].reshape(1,-1))[0]

1

In [58]:
ress = []
for i in tqdm(range(10)):
    res = clf.predict(X_test[100*i:100*(i+1)].reshape(100,-1))
    #print(res,' ',y_ver[i])
    #ress.append(res)
    print(res+1)

 10%|████████▎                                                                          | 1/10 [00:02<00:21,  2.44s/it]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0
 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 2 0 0]


 20%|████████████████▌                                                                  | 2/10 [00:05<00:20,  2.51s/it]

[0 0 0 0 0 1 1 0 0 0 0 0 1 0 1 0 1 0 0 0 1 1 1 0 0 0 2 0 0 0 0 1 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 1 0 2 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0]


 30%|████████████████████████▉                                                          | 3/10 [00:07<00:17,  2.56s/it]

...

[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0]


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [00:22<00:02,  2.46s/it]

[0 1 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0
 0 1 1 0 0 1 0 0 0 0 0 2 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:25<00:00,  2.54s/it]

[0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 2 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 2 0 0 0 0 0 1 0 0 0 0]


In [26]:
ress = []
for i in tqdm(range(504891)):
    res = clf.predict(X_test[i].reshape(1,-1))[0]
    #print(res,' ',y_ver[i])
    ress.append(res)

100%|████████████████████████████████████████████████████████████████████████| 504891/504891 [2:48:50<00:00, 49.84it/s]


In [34]:
ans = np.array(ress)+1

In [29]:
len(ress)

504891

In [39]:
ans_1 = test[['link','current_slice_id','future_slice_id','label']]

In [45]:
ans_1

,link,current_slice_id,future_slice_id,label
0,106984,7,7,-1
1,33252,3,4,-1
...,...,...,...,...
504889,359102,4,4,-1
504890,226212,4,4,-1


In [46]:
ans_1['label'] = ans

In [47]:
ans_1

,link,current_slice_id,future_slice_id,label
0,106984,7,7,1
1,33252,3,4,1
...,...,...,...,...
504889,359102,4,4,1
504890,226212,4,4,1


In [48]:
ans_1.to_csv('2020_10_31.csv', index=False, encoding='utf8')

In [49]:
df_test = pd.read_csv(path, sep=';', header=None)

In [58]:
ahh = df_test[['link','current_slice_id','future_slice_id','label']]

In [59]:
ahh.to_csv('2020_10_31.csv', index=False, encoding='utf8')

In [55]:
df_test['label'] = ans

In [56]:
df_test

,0,1,2,3,4,5,link,label,current_slice_id,future_slice_id
0,106984 -1 616 640,"612:28.10,36.80,1,21 613:30.20,36.80,1,19 614:...","640:28.80,40.90,1,18 641:37.80,41.20,1,21 642:...","640:29.10,39.50,1,37 641:30.00,40.50,1,41 642:...","640:30.30,37.40,1,37 641:33.80,37.70,1,38 642:...","640:30.80,36.20,1,39 641:31.60,36.30,1,32 642:...",106984,1,616,640
1,33252 -1 327 343,"323:23.30,23.00,2,2 324:22.90,22.40,2,1 325:22...","343:28.30,25.10,1,3 344:27.10,25.10,1,3 345:27...","343:32.20,24.10,1,5 344:31.20,24.50,1,6 345:34...","343:33.20,33.30,1,5 344:33.60,29.30,1,5 345:29...","343:31.70,19.10,1,4 344:31.70,20.80,1,5 345:32...",33252,1,327,343
...,...,...,...,...,...,...,...,...,...,...
504889,359102 -1 451 459,"447:52.50,47.30,1,103 448:48.70,44.00,1,106 44...","459:43.90,41.30,1,94 460:45.50,40.80,1,107 461...","459:49.40,38.30,1,120 460:46.80,40.30,1,118 46...","459:43.50,39.20,1,111 460:43.60,40.90,1,113 46...","459:48.70,46.40,1,107 460:45.30,45.20,1,96 461...",359102,1,451,459
504890,226212 -1 378 385,"374:34.90,48.40,1,2 375:28.80,19.90,1,2 376:28...","385:40.60,44.20,1,2 386:38.50,44.20,1,2 387:38...","385:34.80,38.80,1,4 386:36.70,52.60,1,2 387:40...","385:35.70,29.30,1,1 386:29.90,25.50,1,2 387:37...","385:34.80,34.80,1,2 386:34.80,34.80,1,2 387:30...",226212,1,378,385


In [53]:
df_test['link'] = df_test[0].apply(lambda x: x.split(' ')[0])
df_test['label'] = -1
df_test['current_slice_id'] = df_test[0].apply(lambda x: int(x.split(' ')[2]))
df_test['future_slice_id'] = df_test[0].apply(lambda x: int(x.split(' ')[3]))